In [0]:
!git clone https://github.com/datamllab/rlcard.git

In [0]:
%cd rlcard

In [0]:
%pip install -e .

In [0]:
%pip install rlcard[torch]

In [0]:
%pip install rlcard[tensorflow]

In [0]:
import tensorflow as tf
import os

import rlcard
from rlcard.agents.dqn_agent import DQNAgent
from rlcard.agents.random_agent import RandomAgent
from rlcard.utils.utils import set_global_seed, tournament
from rlcard.utils.logger import Logger

In [0]:
# Make environment
env = rlcard.make('mahjong')
eval_env = rlcard.make('mahjong')

# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 100
evaluate_num = 1000
episode_num = 100000
# episode_num = 5000

# The intial memory size
memory_init_size = 1000

# Train the agent every X steps
train_every = 1

# The paths for saving the logs and learning curves
log_dir = './experiments/mahjong_dqn_result/'

# Set a global seed
set_global_seed(0)

with tf.Session() as sess:

    # Initialize a global step
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # Set up the agents
    agent = DQNAgent(sess,
                     scope='dqn',
                     action_num=env.action_num,
                     replay_memory_size=20000,
                     replay_memory_init_size=memory_init_size,
                     train_every=train_every,
                     state_shape=env.state_shape,
                     mlp_layers=[512,512])
    random_agent = RandomAgent(action_num=eval_env.action_num)
    env.set_agents([agent, random_agent, random_agent, random_agent])
    eval_env.set_agents([agent, random_agent, random_agent, random_agent])

    # Initialize global variables
    sess.run(tf.global_variables_initializer())

    # Init a Logger to plot the learning curve
    logger = Logger(log_dir)

    for episode in range(episode_num):
        print("************************Episode:", episode)
        # Generate data from the environment
        trajectories, _ = env.run(is_training=True)

        # Feed transitions into agent memory, and train the agent
        for ts in trajectories[0]:
            agent.feed(ts)

        # Evaluate the performance. Play with random agents.
        if episode % evaluate_every == 0:
            logger.log_performance(env.timestep, tournament(eval_env, evaluate_num)[0])

    # Close files in the logger
    logger.close_files()

    # Plot the learning curve
    logger.plot('DQN')
    
    # Save model
    save_dir = 'models/mahjong_dqn'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    saver = tf.train.Saver()
    saver.save(sess, os.path.join(save_dir, 'model'))

Streaming output truncated to the last 5000 lines.
INFO - Agent dqn, step 563590, rl-loss: 0.0003838805132545531************************Episode: 23636
INFO - Agent dqn, step 563614, rl-loss: 0.00046555796870961785************************Episode: 23637
INFO - Agent dqn, step 563640, rl-loss: 0.0006087635410949588************************Episode: 23638
INFO - Agent dqn, step 563664, rl-loss: 0.0008535417728126049************************Episode: 23639
INFO - Agent dqn, step 563688, rl-loss: 0.00039133295649662614************************Episode: 23640
INFO - Agent dqn, step 563712, rl-loss: 0.0001046617835527286************************Episode: 23641
INFO - Agent dqn, step 563736, rl-loss: 0.00011358059418853372************************Episode: 23642
INFO - Agent dqn, step 563761, rl-loss: 0.00013588982983492315************************Episode: 23643
INFO - Agent dqn, step 563786, rl-loss: 0.00040678560617379844************************Episode: 23644
INFO - Agent dqn, step 563812, rl-loss: 0.00

KeyboardInterrupt: ignored

In [0]:
import tensorflow as tf
import os

import rlcard
from rlcard.agents.nfsp_agent import NFSPAgent
from rlcard.agents.random_agent import RandomAgent
from rlcard.utils.utils import set_global_seed, tournament
from rlcard.utils.logger import Logger

In [0]:
# Make environment
env = rlcard.make('mahjong')
eval_env = rlcard.make('mahjong')

# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 1000
evaluate_num = 1000
episode_num = 5000

# The intial memory size
memory_init_size = 1000

# Train the agent every X steps
train_every = 64

# The paths for saving the logs and learning curves
log_dir = './experiments/mahjong_nfsp_result/'

# Set a global seed
set_global_seed(0)

with tf.Session() as sess:

    # Initialize a global step
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # Set up the agents
    agents = []
    for i in range(env.player_num):
        agent = NFSPAgent(sess,
                          scope='nfsp' + str(i),
                          action_num=env.action_num,
                          state_shape=env.state_shape,
                          hidden_layers_sizes=[512,1024,2048,1024,512],
                          anticipatory_param=0.5,
                          batch_size=256,
                          rl_learning_rate=0.00005,
                          sl_learning_rate=0.00001,
                          min_buffer_size_to_learn=memory_init_size,
                          q_replay_memory_size=int(1e5),
                          q_replay_memory_init_size=memory_init_size,
                          train_every = train_every,
                          q_train_every=train_every,
                          q_batch_size=256,
                          q_mlp_layers=[512,1024,2048,1024,512])
        agents.append(agent)
    random_agent = RandomAgent(action_num=eval_env.action_num)

    env.set_agents(agents)
    eval_env.set_agents([agents[0], random_agent, random_agent, random_agent])

    # Initialize global variables
    sess.run(tf.global_variables_initializer())

    # Init a Logger to plot the learning curvefrom rlcard.agents.random_agent import RandomAgent

    logger = Logger(log_dir)

    for episode in range(episode_num):

        # First sample a policy for the episode
        for agent in agents:
            agent.sample_episode_policy()

        # Generate data from the environment
        trajectories, _ = env.run(is_training=True)

        # Feed transitions into agent memory, and train the agent
        for i in range(env.player_num):
            for ts in trajectories[i]:
                agents[i].feed(ts)

        # Evaluate the performance. Play with random agents.
        if episode % evaluate_every == 0:
            logger.log_performance(env.timestep, tournament(eval_env, evaluate_num)[0])

    # Close files in the logger
    logger.close_files()

    # Plot the learning curve
    logger.plot('NFSP')
    
    # Save model
    save_dir = 'models/mahjong_nfsp'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    saver = tf.train.Saver()
    saver.save(sess, os.path.join(save_dir, 'model'))

In [0]:
import numpy as np

import rlcard
from rlcard.agents.cfr_agent import CFRAgent
from rlcard.agents.random_agent import RandomAgent
from rlcard import models
from rlcard.utils.utils import set_global_seed, tournament
from rlcard.utils.logger import Logger

In [0]:
# # Make environment and enable human mode
# env = rlcard.make('mahjong', config={'allow_step_back':True})
# eval_env = rlcard.make('mahjong')

# # Set the iterations numbers and how frequently we evaluate the performance and save model
# evaluate_every = 100
# save_plot_every = 1000
# evaluate_num = 10000
# episode_num = 10000


# # The intial memory size
# memory_init_size = 1000

# # Train the agent every X steps
# train_every = 64

# # The paths for saving the logs and learning curves
# log_dir = './experiments/mahjong_cfr_result/'

# # Set a global seed
# set_global_seed(0)

# # Set up the agents
# agents = []
# for i in range(env.player_num):
#     # Initilize CFR Agent
#     agent = CFRAgent(env)
#     # If we have saved model, we first load the model
#     agent.load()
#     agents.append(agent)
# random_agent = RandomAgent(action_num=eval_env.action_num)

# env.set_agents(agents)
# eval_env.set_agents([agents[0], random_agent, random_agent, random_agent])

# # Init a Logger to plot the learning curve
# logger = Logger(log_dir)

# for episode in range(episode_num):
#     agent.train()
#     print('\rIteration {}'.format(episode), end='')
#     # Evaluate the performance. Play with NFSP agents.
#     if episode % evaluate_every == 0:
#         agent.save() # Save model
#         logger.log_performance(env.timestep, tournament(eval_env, evaluate_num)[0])

# # Close files in the logger
# logger.close_files()

# # Plot the learning curve
# logger.plot('CFR')

# # Save model
# save_dir = 'models/mahjong_cfr'
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)
# saver = tf.train.Saver()
# saver.save(sess, os.path.join(save_dir, 'model'))

# Make environment and enable human mode
# env = rlcard.make('mahjong', config={'allow_step_back':True})
# eval_env = rlcard.make('mahjong')

# # Set the iterations numbers and how frequently we evaluate the performance and save model
# evaluate_every = 100
# save_plot_every = 1000
# evaluate_num = 10000
# episode_num = 10000

# # The paths for saving the logs and learning curves
# log_dir = './experiments/mahjong_cfr_result/'

# # Set a global seed
# set_global_seed(0)

# # Initilize CFR Agent
# agent = CFRAgent(env)
# agent.load()  # If we have saved model, we first load the model
# random_agent = RandomAgent(action_num=eval_env.action_num)

# # Evaluate CFR against pre-trained NFSP
# eval_env.set_agents([agent, random_agent, random_agent, random_agent])

# # Init a Logger to plot the learning curve
# logger = Logger(log_dir)

# for episode in range(episode_num):
#     agent.train()
#     print('\rIteration {}'.format(episode), end='')
#     # Evaluate the performance. Play with NFSP agents.
#     if episode % evaluate_every == 0:
#         agent.save() # Save model
#         logger.log_performance(env.timestep, tournament(eval_env, evaluate_num)[0])

# # Close files in the logger
# logger.close_files()

# # Plot the learning curve
# logger.plot('CFR')